In [1]:
import itertools
import requests
import hashlib
import os
import re
import collections
from functools import reduce

## Search Functions
For two different problems I thought I would need a BFS / DFS but didn't end up needing them.  I did however, waste a lot of time debugging them since I'm coding them from memory and they are easy to introduce subtle bugs into (at least for me). Since I'm fairly sure I'll need some search at some point, I'm just going to leave them here for future use (to be expanded as necessary, e.g. I'm not going to bother with `A*` until I need to).

In [2]:
def bfs(start,goal,successors = lambda x: x):
    frontier = collections.deque([[start]])
    seen = set(tuple(start))
    while frontier:
        path = frontier.popleft()
        node = path[-1]
        if tuple(node) not in seen:
            seen.add(tuple(node))
            for s in successors(node):
                if s == goal:
                    return path,len(path)
                frontier.append(path + [s])
    return seen

def dfs(start,goal,successors = lambda x: x):
    frontier = [[start]]
    seen = set(tuple(start))
    while frontier:
        path = frontier.pop()
        node = path[-1]
        if tuple(node) not in seen:
            seen.add(tuple(node))
            for s in successors(node):
                if s == goal:
                    return path,len(path)
                frontier.append(path + [s])
    return seen

# Day One
This should be pretty simple... for part 1 all you need to do is compare adjacent items up until the length of the list minus 1 (minus 1 because at the end, or second to last element, we compare to  i + 1).  Since I'm going for time, rather than spend the extra 30 seconds writing the code to handle when the index goes beyond the list, I just eyeballed it and saw the last element was the same as the first, and added it.


In [3]:
data = open('day1.txt').read().strip()


## part one

In [4]:
sum([int(x) for x,y in (zip(data, data[1:])) if x == y]) + 3 #cheated :)

1069

## part two

In [5]:
size = len(data) // 2

In [6]:
res = []
for i in range(len(data)):
    ix = min(i + size,i + size - len(data))
    if data[i] == data[ix]:
        res.append(int(data[i]))

In [7]:
sum(res)

1268

# Day Two
Our input for day two is a series of rows containing tab delimited numbers, meant to mimic a spreadsheet layout.  This is straight-forward to parse by simply splitting on newlines to get the rows, and then doing a regex on each row to get numbers out, rather than dealing with the tabs. Once this is done, you still have to convert each text input to an integer in order to perform the arithmetic.

In [144]:
data2 = open('day2.txt').read().strip()

In [145]:
rows = data2.split('\n')

## part one
Part one asks us to find the difference of the minimum and maximum of each row, and then sum all of those values to calculate the "checksum" which will be our answer.  

In [146]:
numrows = [list(map(int,re.findall('\d+',row))) for row in rows]

In [147]:
sum(max(row) - min(row) for row in numrows)

37923

## part two
Part two requires us to find the only two evenly divisible items in each row, calculate the value of this division, and then sum the results.  Itertools to the rescue.... I'll just get all combinations of size 2 for each row and test divisibility

In [148]:
sum(max(x,y) / min(x,y) for row in numrows 
                        for x,y in itertools.combinations(row,2) 
                        if max(x,y) % min(x,y) == 0)

263.0

# Day 3 
The minute I read this puzzle I may have mumbled a few choice words to myself.  I've seen something similar before on project euler and punted on it, because I'm not particularly great at the more "mathy" or number theory-esque problem types.  But, since this is Advent of Code, and I'm fully committed to at least doing my very best to solve all 25 days, I'm gonna have to buckle down and try to figure it out.

## Part One
This is my WIP for part one, I havent even gotten to part two.  Day four has come and gone and was fortunately way easiser. I'm still mulling this over and trying to find a solution without looking at others because I feel I'm getting close since I've found the recurrence relation, I just need to figure out how to use that to calculate the distance to the target.

<strong>UPDATE</strong>: After basically an entire day of kicking around ideas and ironing out some bugs, I got a working solution for part 1, a bit before day 5 was released.  The code is most likely overly complicated which tends to be the case when I don't have a clear idea how to solve a problem and I'm just iteratively building on ideas until I get to a solution.  Below the solutions to part one and two are some properties I found which I had originally hoped to use to solve part one (a recurrence relation) but in the end couldn't figure out how to get a solution using this approach.

I was a bit nervous on what part two was going to be like since most people seemed to think that was much more difficult than the first part, but it turned out to be fairly straight-foward to extend my part one code to solve part two.  I guess I got lucky in the way I implemented part one - my guess is alot of people found a nice analytical solution to part one and then had to scrap and start from nothing for part two.  My lack of math skills saved me from the same folly since I basically had constructed the grid already in part one, which made part two much simpler.

In [15]:
import math

loc = [0,0]
target = 265149
MOVES = [[0,1],[-1,0],[0,-2],[2,0],[0,2]]
INCREMENTS = [[0,0],[-2,0],[0,-2],[2,0],[0,2]]
moves_so_far = 1

def vector_process(v1,v2,func=lambda x,y: x+y):
    return [func(a,b) for a,b in zip(v1,v2)]

def move(loc,move,moves_so_far):
    new_loc = vector_process(loc,move)
    moves_so_far += sum([abs(num) for num in move])
    return new_loc,moves_so_far

def manhattan_distance(c1,c2):
    return sum(vector_process(c1,c2,func=lambda x,y: abs(x-y)))

def cumulative_move_steps(move):
    x,y = move 
    moves = []
    if x < 0:
        for i in range(-1,x-1,-1):
            moves.append([-1,y])
    elif x > 0:
        for i in range(1,x+1):
            moves.append([1,y])
    elif y < 0:
        for i in range(-1,y-1,-1):
            moves.append([x,-1])
    elif y > 0:
        for i in range(1,y+1):
            moves.append([x,1])
    return moves

def part1():
    global MOVES,loc,moves_so_far
    while True:
        for m in MOVES:
            for c in cumulative_move_steps(m):
                loc,moves_so_far = move(loc,c,moves_so_far)
                if moves_so_far >= target:
                    return manhattan_distance(loc,[0,0])
        MOVES =  [vector_process(m[0],m[1]) 
                  for m in zip(MOVES,INCREMENTS)]


def make_grid(row,col):
    return [[0 for i in range(col)] for j in range(row)]
print(part1())



438


In [16]:
def part2(grid):
    MOVES = [[0,1],[-1,0],[0,-2],[2,0],[0,2]]
    cur_number = 1
    grid_size = len(grid)
    center_x,center_y = loc = [grid_size // 2] * 2
    grid[center_x][center_y] = 1
    def move(loc,move,cur_number):
        new_loc = vector_process(loc,move)
        cur_number = sum(grid[x][y] for x,y in neighbors(new_loc))
        return new_loc,cur_number
    def neighbors(loc):
        return [vector_process(loc,(i,j)) for i in range(-1,2)
                                          for j in range(-1,2) 
                                          if (0 <= i + loc[0] < grid_size and 0 <= j + loc[1] < grid_size)
                                          and (i,j) != (0,0)]
    while True:
        for m in MOVES:
            for c in cumulative_move_steps(m):
                loc,cur_number = move(loc,c,cur_number)
                grid[loc[0]][loc[1]] = cur_number
                if cur_number > target:
                # return manhattan_distance(loc,[center_x,center_y]) RTFM
                    return cur_number
        MOVES =  [vector_process(m[0],m[1]) 
                  for m in zip(MOVES,INCREMENTS)]


print(part2(make_grid(515,515)))

266330


The following were my original attempts at solving the puzzle before I threw in the towel and slept on it.  The next day I still could'nt figure out how to use these to solve part 1 so I just caved and chose to build the grid as seen above.  I still got some satisfaction out of this though, because I really like recursion and always feel recursive solutions are elegant and beautiful :)

In [17]:
diags = [3,9,7,5]
lrud =  [2,4,6,8]  

def gen_spiral(n,target = target):
    n = n
    d = n - 1
    steps = 1
    nums = [n]
    while n < target:
        d += 8
        n += d
        steps += 1
        nums.append(n)
    return nums
    
for d in diags:
    spirals = gen_spiral(d)
    if target in spirals:
        print(spirals.index(target) * 2)
        break
for x in lrud:
    spirals = gen_spiral(x)
    if target in spirals:
        print(spirals.index(target))
        break
            


In [18]:
print(gen_spiral(6)[0:10])

[6, 19, 40, 69, 106, 151, 204, 265, 334, 411]


In [19]:
def gen_spiral(n,prevn):
    yield n
    d = n - prevn + 8
    n += d
    if n < target: yield from gen_spiral(n,n-d)

print(list(gen_spiral(24,9))[0:10])

[24, 47, 78, 117, 164, 219, 282, 353, 432, 519]


In [20]:
def spiral(n):
    return 1 if n <= 0 else 2 * spiral(n-1) - spiral(n-2) + 8

list(map(spiral,range(0,10)))

[1, 9, 25, 49, 81, 121, 169, 225, 289, 361]

# Day 4
Here the input is a colleciton of strings delimited by newlines.  Within each string, or "phrase", we need to see if there are any duplicates and filter those out, and then count the remaining "phrases".

## Part One

In [21]:
data = open('day4.txt').read().strip()
words = data.split('\n')

In [22]:
valid = [phrase for phrase in words if len(set(phrase.split())) == len(phrase.split())]
print(len(valid))

325


## Part Two
To get anagrams of a word I just found all permutations of the word in question, and then checked the rest of the list for each permutation.  This can be wasteful since you end up calculating unnecessary permutations - for example, if your list is 

['Christian','Christina']

Only the last two letters have changed, so you should not consider any permutations where any of the first seven letters differ.  You can do this by creating a list of valid prefixes of your candidates, and making sure to not progress along the permutaitons unless your current permutation is seen in that candidate prefix list.  

Because the amount of data I'm dealing with is small, and I am trying to move quickly, I'm not going to prematurely optimize and just brute force it by calculating all permutations, regardless of whether or not they are potentially invalid early on.

In [23]:
def anagrams(word):
    return {''.join(a) for a in (itertools.permutations(word))}


In [24]:
def anagram_found(text):
    words = text.split()
    for i in range(len(words) - 1):
        for a in anagrams(words[i]):
            if a in words[i+1:]:
                return True
    return False
            

In [25]:
assert(not(anagram_found('abcde fghij')))
assert(anagram_found('abcde xyz ecdab'))
assert(not(anagram_found('a ab abc abd abf abj')))
assert(not(anagram_found('iiii oiii ooii oooi oooo')))
assert(anagram_found('oiii ioii iioi iiio'))

In [26]:
phrases = [phrase for phrase in words if not(anagram_found(phrase))]
len(phrases)

119

In [27]:
phrases[0:5]

['ojufqke gpd olzirc jfao cjfh rcivvw pqqpudp',
 'wchrl pzibt nvcae wceb',
 'rdwytj kxuyet bqnzlv nyntjan dyrpsn zhi kbxlj ivo',
 'qwx ubca dxudny oxagv wqrv lhzsl qmsgv dxs awbquc akelgma',
 'rrdlfpk ohoszz qiznasf awchv qnvse']

# Day 5
Day 5 reminds me of some of the problems from last year where you had to build a mini interpreter for a limited set of machine instructions, but on a smaller scale.  The instructions turned out to be fast enough without trying to find a patterm or optimizing the sequence by some form of pre-processing, but I wouldn't be surprised if, like last year, later puzzles build on the sequence and make it prohitively slow without some optmizations.  

As presented, it's fairly easy to solve using some globals to keep state, and just bailing once the position is outside of the list.  I do wonder how I'll approach this in a more functional style, since I'm trying to do all of these problems in Clojure once I've finished them in Python first.  Typically whenever I think `while` in an imperative language, I think `reduce` in a functional language, so I'll probably start there.

In [28]:
data = open('day5.txt').read().strip().split('\n')
nums = [int(x) for x in data]

## Part One

In [29]:
pos = 0
cnt = 0
while pos in range(len(nums)):
    offset = nums[pos]
    nums[pos] += 1
    pos += offset
    cnt += 1
print(cnt)    

381680


## Part Two

In [30]:
nums = [int(x) for x in data] #rebuild nums since we mutated it in step 1

pos = 0
cnt = 0
while pos in range(len(nums)):
    offset = nums[pos]
    if offset >= 3:
        nums[pos] -=1
    else:
        nums[pos] +=1
    pos += offset
    cnt += 1
print(cnt)    

29717847


# Day 6
This puzzle involves taking a sequence of numbers, or "memory banks", each of which has "n" blocks.  We are asked to find how many cycles it takes until any sequence is seen a second time, where a cycle is defined as follows:
- find the maximum of the sequence, ties are by the first seen (lowest index)
- reduce the blocks at this location to zero, and spread that amount over all of the subsequent banks (list elements), wrapping around once the end is hit.
- return the number of cycles that have occured once a repeat is hit.

In [31]:
nums = [10,3,15,10,5,15,5,15,9,2,5,8,5,2,3,6]

## Part One
I started down the path of a recursive solution, because that seemed the most natural to me at the time.  Just make the updates to the list, add the result to the `seen` set, and recur with an incremented count.  Except I forgot about the 1,000 depth recursion limit in Python.  I guess this is what happens when you spend too much time in Clojure lately.  I burned a good 20 minutes trying to manually increase the recursion limit using `sys.setrecursionlimit(n)` and kept getting an infuriating error `TypeError: 'int' object is not callable`.  I eventually gave up since this isn't the documented behavior and I couldn't figure out why it was failing.  In any event, I probably should not treat an iterative problem with a recursive solution in a language without tail call optmization, so I just converted the solution to a nested `while` loop instead.

Here is my recursive version, which worked on the small test input and then exploded on the actual input when the limit surpassed 1,000:

In [32]:
def cycle(nums,res=set(),cnt=1):
    highest = max(nums)
    loc = nums.index(highest)
    nums[loc] = 0
    while highest > 0:
        loc += 1
        if loc > len(nums) - 1:
            loc =  loc % (len(nums))
        nums[loc] += 1
        highest -= 1
    if tuple(nums) in res:
        return nums,cnt
    res.add(tuple(nums))
    return cycle(nums,res,cnt+1)

and here is the version I had to replace it with after giving up on changing the recursion limit (which is obviously bad form, but I had already invested time in a working solution and wanted to not burn more time by re-writing (which didn't really take that much effort in the end anyhow).  Something about this solution rubs me the wrong way, it doesn't feel that elegant but it gets the job done.  I think it's the nested while loops that are giving me that feeling.  Anyhow - when I'm trying to solve these fast I focus on just getting something working first, making it pretty comes later.  Maybe I'll clean all these up and show the first version and the second polished version when I have more time.

In [33]:
nums = [10,3,15,10,5,15,5,15,9,2,5,8,5,2,3,6] #rebind since mutated above
seen = set()
cnt = 1
while True:
    highest = max(nums)
    loc = nums.index(highest)
    nums[loc] = 0
    while highest > 0:
        loc += 1
        if loc > len(nums) - 1:
            loc =  loc % (len(nums))
        nums[loc] += 1
        highest -= 1
    if tuple(nums) in seen:
        print(nums,cnt)
        break
    seen.add(tuple(nums))
    cnt += 1


[1, 1, 0, 15, 14, 13, 12, 10, 10, 9, 8, 7, 6, 4, 3, 5] 14029


## Part Two
Here the problem is modified such that instead of finding the cycles, we find the distance between cycles.  E.G. if the first occurence was on our third cycle, and the repeat of that occurence was at the 10th cycle, the distance is seven.  The only modificaton I need to make is swapping out my `seen` set for a dictionary, with the key as the actual sequence (tuple so it's hashable), and the value as the cycle it occured in.  On repeat I just find the current count minus the previous count, which i look up using the current sequence.

In [34]:
nums = [10,3,15,10,5,15,5,15,9,2,5,8,5,2,3,6] #rebind since mutated above
seen = dict()
cnt = 1
while True:
    highest = max(nums)
    loc = nums.index(highest)
    nums[loc] = 0
    while highest > 0:
        loc += 1
        if loc > len(nums) - 1:
            loc =  loc % (len(nums))
        nums[loc] += 1
        highest -= 1
    if tuple(nums) in seen:
        print(nums,cnt - seen[tuple(nums)])
        break
    seen[tuple(nums)] = cnt
    cnt += 1



[1, 1, 0, 15, 14, 13, 12, 10, 10, 9, 8, 7, 6, 4, 3, 5] 2765


# Day Seven
This puzzle is about trees and recursion - two things that typically go well together.  We are given a tree in the form of an adjacency list, but we don't know where the root of the tree is - our task is to find it.  Typically when traversing a tree, you start at the root and either recursively evaluate children (for depth first search), or maintain a queue and evaluate children left to right (for breadth first search)


## Part One

I found this problem a little tricky - typically when working with tree or graph structures I know where I'm starting, here I had to think about how I would find that start if I didn't know.  The path I chose to go down (no pun intended) was to invert the tree, and create mappings from children-to-parents instead of the given parent-to-children structure.  This means that if the input had a parent `A -> ['B','C','C']` I would create three entires in my reverse tree for each child, all of which mapped to the parent value `A`.  

Once that's done, I picked a random node ('dzzbkv' below) and started working my way up the parents, until their weren't any.  I have a feeling this might not be the best way to solve it, but it worked for me.

In [35]:
data = open('day7.txt').read().strip().split('\n')

In [36]:
nodes = [re.findall('\w+',node) for node in data]
tree = {n[0]:n[2:] for n in nodes}
reverse_tree = collections.defaultdict(list)
for k,v in tree.items():
    for node in v:
        reverse_tree[node].append(k)

In [37]:
def traverse(tree,start='dzzbvkv'):
    """pick a random start in the reversed tree, continue until nowhere to go"""
    if tree[start]:
        for n in tree[start]:
            return traverse(tree,n)
    return start
traverse(reverse_tree)

'hlhomy'

## Part Two
Part two is tricky for me - it's still tree traversal, but you now have to incorporate the "weights" present at each node.  The way you incorporate them takes a bit of explaining so rather than regurgitate it here, just read the instructions on Advent of Code.

In short, however, the idea is to make sure that the cumulative values of each child node of a parent are all equal, and therefore the parent is balanced.  One node is not balanced, and we need to find what it's value should be in order to make it balanced. Cumulative means that a node's value is the sum of it's children's values, plus it's own.

In [38]:
data = open('day7.txt').read().strip().split('\n')
tree = [[list(re.match('(\w+) \((\d+)\)',x[0]).groups()),x[1:]]
        for x in [line.split('->') for line in data]]

new_tree = {}

for node, neighbors in tree:
    val = int(node.pop())
    if neighbors:
        neighbors = [n.strip() for n in neighbors[0].strip().split(',')]
    new_tree[node[0]] = [val, neighbors]

root = 'hlhomy'

def traverse(tree,node=root):
    __, children = tree[node]
    if not children: 
        return __
    for c in children:
        val, children = tree[node]
        # if found: return 
        tree[node][0] = traverse(tree,c) + val
    __ , children = tree[node]
    cvals = [tree[c][0] for c in children]
    if len(set(cvals)) == 1:
        return tree[node][0]
    else:
        unbalanced_value   = [c for c in cvals if cvals.count(c) == 1][0]
        val_it_should_be   = [c for c in cvals if cvals.count(c) != 1][0]
        unbalanced_node    = [c for c in children if tree[c][0] == unbalanced_value][0]
        __, unbalanced_nodes_children = tree[unbalanced_node]
        unbalanced_node_values = [tree[c][0] for c in unbalanced_nodes_children]
        val_to_balance  = val_it_should_be - sum(unbalanced_node_values)
        print(val_it_should_be,sum(unbalanced_node_values),val_to_balance)
        return  val_to_balance

print(traverse(new_tree))

1571 66 1505
22313 6292 16021
111630 111573 57
57


# Day Eight
Another mini-interpreter style problem.  This one is easy to parse, and the instructions are fairly limited.  I am almost certain we'll see a similar puzzle later with more involved parsing, and a greater set of operations to choose from.  Part 1 and 2 are both really solved by the same piece of code below, the only difference is that part 1 asks for the maximum register value *after* all instructions have run, while part 2 asks for the maximum value ever seen.

## Part One

In [39]:
data = open('day8.txt').read().strip().split('\n')

In [40]:
instructions = [re.match('(\w+) (inc|dec) (-*\d+) if (.*)',instr).groups() for instr in data]

In [41]:
registers = {instr[0]:0 for instr in instructions}

def process_expr(expr):
    reg, test, val = expr.split(' ')
    reg_val = registers[reg]
    expr = (''.join(str(reg_val) + test  + str(val)))
    return eval(expr)

greatest_so_far = 0

for reg,op,val,expr in instructions:
    current_reg_val = registers[reg]
    if process_expr(expr):
        if op == 'inc':
            registers[reg] = current_reg_val + int(val)
        else:
            registers[reg] = current_reg_val - int(val)
    current_highest_val = max(registers.values())
    if current_highest_val > greatest_so_far:
        greatest_so_far = current_highest_val


In [42]:
print(greatest_so_far)

6696


## Part Two
Processing the instruction set and making the indicated modifications to the registers results in solving both problems at once - although I didn't know this until I read part two.  After going through all the instructions and changing the register values, part one is just the max of the register values, and the only change I needed to make for part two was adding the global `greatest_so_far` and the `if current_highest_val > greatest_so_far` test at the bottom

In [43]:
print(max(registers.values()))

6061


## Day Nine
I'm noticing a theme here... lot's of recursion and interpreters.  I tried to use recursion but got bit again by python's limit sinc the input text is much larger than 1,000, and had to re-write to some nested while loops, yet again.  I included the recursive version below anyhow, since I had already buit it on the tests cases before figuring out the input was too large for it to work.

Part one and two are both solved below, there is only a slight modification for part two.

In [44]:
data = open('day9.txt').read().strip()

In [45]:
def process_stream(s,val=1,garbage=False):
    if not s: return 0
    first, rest = s[0], s[1:]
    if first == '{':
        if not garbage:
            return val + process_stream(rest,val + 1,garbage)
        else:
            return process_stream(rest,val,garbage)
    elif first == '}':
        if not garbage:
            return process_stream(rest, val - 1,garbage)
        else:
            return process_stream(rest, val,garbage)
    elif first == '!':
        return process_stream(rest[1:],val,garbage)
    elif first == '<':
        return process_stream(rest,val,garbage=True)
    elif first == '>':
        return process_stream(rest,val,garbage=False)
    else:
        return process_stream(rest,val,garbage)
        
# print(process_stream(data))   #text > 1,000, over python's recursion limit :(

def test():
    assert(process_stream('{}') == 1)
    assert(process_stream('{{{}}}') == 6)
    assert(process_stream('{{},{}}') == 5)
    assert(process_stream('{{{},{},{{}}}}') == 16)
    assert(process_stream('{<a>,<a>,<a>,<a>}') == 1)
    assert(process_stream('{{<ab>},{<ab>},{<ab>},{<ab>}}') == 9)
    assert(process_stream('{{<!!>},{<!!>},{<!!>},{<!!>}}') == 9)
    assert(process_stream('{{<a!>},{<a!>},{<a!>},{<ab>}}') == 3)
    return "tests pass"
test()

'tests pass'

In [46]:
def process_stream_non_recursive(s,part=1):
    val = 0
    ix = 0
    totals = []
    garbage_count = 0
    while ix < len(s):
        if s[ix] == '{':
            val += 1
            totals.append(val)
        elif s[ix] == '}':
            val -= 1
        elif s[ix] == '<':
            ix += 1
            while ix < len(s):
                if s[ix] == '!':
                    ix += 2
                elif s[ix] == '>':
                    break
                else:
                    garbage_count += 1
                    ix += 1
        ix += 1
    if part == 1:
        return sum(totals) #part one
    else:
        return garbage_count #part two

def test_day9():
    assert(process_stream_non_recursive('{}') == 1)
    assert(process_stream_non_recursive('{{{}}}') == 6)
    assert(process_stream_non_recursive('{{},{}}') == 5)
    assert(process_stream_non_recursive('{{{},{},{{}}}}') == 16)
    assert(process_stream_non_recursive('{<a>,<a>,<a>,<a>}') == 1)
    assert(process_stream_non_recursive('{{<ab>},{<ab>},{<ab>},{<ab>}}') == 9)
    assert(process_stream_non_recursive('{{<!!>},{<!!>},{<!!>},{<!!>}}') == 9)
    assert(process_stream_non_recursive('{{<a!>},{<a!>},{<a!>},{<ab>}}') == 3)
    print("tests pass.")
test_day9()    

tests pass.


## Part One

In [47]:
process_stream_non_recursive(data,part=1)

14212

## Part Two

In [48]:
process_stream_non_recursive(data,part=2)

6569

# Day Ten
OK... this gets my vote for most obnoxious puzzle so far.  It was conceptually simple, but for me it was extremely tedious to code part 1.  I kept introducing OBO bugs and had a difficult time re-allocating the sorted array back in the main list.  Even though part 2 is considerably more involved, I found it straightforward once part 1 was out of the way.

## Part One

In [49]:
lengths = [189,1,111,246,254,2,0,120,215,93,255,50,84,15,94,62]

In [50]:
def process(data=list(range(0,256)),lengths=lengths):
    pos = 0
    skip = 0
    for l in lengths:
        end = pos + l
        if end > len(data):
            end = end % len(data)
            right, left = data[pos:], data[0:end]
            all_rev = list(reversed(right+left))
            data[pos:] = all_rev[0:len(right)]
            data[0:end] = all_rev[len(data[pos:]):]
        else:
            data = data[0:pos] + list(reversed(data[pos:end])) + data[end:]
        pos = pos + skip + l
        if pos > len(data):
            pos = pos % len(data)
        skip += 1
    return data[0] * data[1] 

In [51]:
process()

38415

## Part Two

In [52]:
stringify = str(lengths)[1:-1].replace(' ','')
to_ascii = list(map(ord,stringify)) + [17, 31, 73, 47, 23]

def process(data=list(range(0,256)),lengths=to_ascii):
    pos = 0
    skip = 0
    for i in range(64):
        for l in lengths:
            end = pos + l
            if end > len(data):
                end = end % len(data)
                right, left = data[pos:], data[0:end]
                all_rev = list(reversed(right+left))
                data[pos:] = all_rev[0:len(right)]
                data[0:end] = all_rev[len(data[pos:]):]
            else:
                data = data[0:pos] + list(reversed(data[pos:end])) + data[end:]
            pos = pos + skip + l
            if pos > len(data):
                pos = pos % len(data)
            skip += 1
    return data

knotted = process()
xord = [knotted[i:i+16] for i in range(0,len(knotted),16)]
xord = [reduce(lambda x, y: x ^ y, segment) for segment in xord]
to_hex = ''.join([hex(x)[2:].zfill(2) for x in xord])
to_hex

'9de8846431eef262be78f590e39a4848'

# Day 11
When I read the description and provided test cases, I had no clue what was going on or how to map from directions to the provided distance test cases.  After some googling, and twitter mentions of redblob games (which I've used extensively in the past for it's ridiculously awesome graph search walk-throughs), I figured out how to move in a hex grid, and then the problem was straight-forward.

That said - I still found the wording confusing.  I interprted the instructions as follows:
 - we are given a path taken by the "child"
 - find the end of the path, or the location the child ends up at
 - find the shortest number of steps to get to that location.
 
Because of the "shortest number of steps" part, I had originally went down the path (no pun intended) of coding a Breadth First Search, after calculating the final location, to find the number of steps to that locaiton.

It turns out all the problem really wanted was the distance from the start of that final location (it already is the shortest path).  You can see this by comparing the results of the BFS below vs the results of simply getting the distance of the end location from the start, or `(0,0,0)`.  They are the same.

## Part One & Two 

In [83]:
data = open('day11.txt').read().strip().split(',')

In [84]:
pos = [0,0,0]
dirs = ['nw','n','ne','sw','s','se']
hex_moves = [(-1,1,0),(0,1,-1),(1,0,-1),(-1,0,1),(0,-1,1),(1,-1,0)]
coord_map = dict(zip(dirs,hex_moves))

def hex_move(moves,start=(0,0,0)):
    max_dist = 0
    for m in moves:
        start = vector_process(coord_map[m],start)
        cur_dist = sum([abs(p) for p in start])/2
        if cur_dist > max_dist:
            max_dist = cur_dist
    return cur_dist,max_dist,start # cur_dist = part 1, max_dist = part 2

# assert(hex_move('ne,ne,ne'.split(',')) ==  3)
# assert(hex_move('ne,ne,sw,sw'.split(',')) == 0) 
# assert(hex_move('ne,ne,s,s'.split(',')) == 2)
# assert(hex_move('se,sw,se,sw,sw'.split(',')) ==  3)

hex_move(data)

(759.0, 1501.0, [445, -759, 314])

In [88]:
path, length = bfs((0,0,0),
                   [445, -759, 314],
                   successors=lambda x: [vector_process(m,x) for m in hex_moves])
print(length)

759


# Day 12
The input is text in the form of `a <-> [b,c,d]`, representing a bi-directional graph as an adjacency list - so `a` can go to `b,c & d`, and the reverse is also true.  We are asked to find the number of "programs contained in the ID of 0", which is basically asking how many unique connections, either directly or indirectly, are there to 0.  

For this I'll use a DFS starting at 0 and just return the length of everything that can be visited starting from that point.

## Part One

In [56]:
data = open('day12.txt').read().strip().split('\n')
data = [prog.split(' <-> ') for prog in data]
data = {int(left):list(map(int,right.split(', '))) for left, right in data}

def dfs(start,successors,cnt=0,visited=set()): # also cnt not incremented bc final stack frame is still 0 when returning up
    if start not in visited:
        visited.add(start)
        for s in successors[start]:
            dfs(s,successors,cnt + 1,visited) # had return here - bug, figure out why tomm
    return len(visited)
            
dfs(0,data)

306

In [57]:
data

{0: [950, 1039],
 1: [317, 904, 923],
 2: [2],
 3: [870],
 4: [361],
 5: [5],
 6: [305, 842, 1932],
 7: [438, 1509],
 8: [985],
 9: [425],
 10: [121, 1532],
 11: [285, 1242, 1753],
 12: [632, 762, 829],
 13: [662, 1429],
 14: [215, 1155, 1523],
 15: [43, 238, 1552],
 16: [1050],
 17: [275],
 18: [26, 256],
 19: [1656],
 20: [1907],
 21: [608, 1493, 1587, 1646],
 22: [1748, 1774],
 23: [723, 871],
 24: [244, 624],
 25: [25],
 26: [18, 490, 545, 1123, 1958],
 27: [27],
 28: [832, 1227],
 29: [285],
 30: [223, 1317, 1773],
 31: [283],
 32: [921],
 33: [395, 1772],
 34: [1170],
 35: [471, 885, 1119],
 36: [936, 1831],
 37: [263, 678, 1356, 1683, 1714],
 38: [1238],
 39: [167],
 40: [804, 1127, 1419],
 41: [365],
 42: [1632, 1788],
 43: [15, 403, 866],
 44: [497, 503, 1305],
 45: [130, 881, 1158],
 46: [46, 744],
 47: [867, 1391, 1630],
 48: [48, 1363],
 49: [1286],
 50: [1687, 1966],
 51: [138, 454, 1105],
 52: [1647],
 53: [762],
 54: [54],
 55: [340, 1913],
 56: [100, 720],
 57: [547, 14

## Part Two
Our twist for part two is that we want to know the total number of unique groups in the graph.  I'll just keep running dfs on every possible node and continually update a master set with all of the group sets generated by traversing each key.  I'm sure there is a better / more efficient way to do this by taking into account what's been seen and not revisiting those keys, but the dumb / brute force way worked fast enough for me, just letting the `seen` set filter out any duplicates.

In [59]:
def dfs(start,successors,cnt=0,visited=set()): # also cnt not incremented bc final stack frame is still 0 when returning up
    if start not in visited:
        visited.add(start)
        for s in successors[start]:
            dfs(s,successors,cnt + 1,visited) # had return here - bug, figure out why tomm
    return visited

seen = set()
for k in data:
    seen.add(frozenset(dfs(k,data)))
print(len(seen))

200


# Day 13
given a series of `layer:depth` integer pairs, we interpet the input as a firewall through which we need to move a packet, while avoiding "scanners".  A single line of input could be 3:5 meaning the third layer of the firewall has a depth of 5.  At each "picosecond" scanners move down one step until they hit the bottom depth, and then they move back up one step until they hit the top, oscillating up and down like an elevator.


## Part One
Given all of the complete firewall, composed of many layer:depth units, we need to find the number of times a packet attempts to *enter* a layer which has a scanner present.  Packets move only along the top of each layer (depth 0).

In [60]:
data = open('day13.txt').read().strip().split('\n')
# data = ['0: 3','1: 2','4: 4','6: 4']
firewalls = [(int(d),'fwd',['s'] + list(range(int(r) - 1))) 
             for (d,r) in [line.split(': ') for line in data]]

valid_depths = {d for d,dr,r in firewalls}
missing_depths = set(range(max(valid_depths) + 1)) - valid_depths
fill_in_missing = [(m,'fwd', ['e']) for m in sorted(missing_depths)]
all_together = sorted(fill_in_missing + firewalls,key = lambda x: x[0])

def elevator(coll,n,ix,dr):
    while n > 0:
        if dr == 'fwd':
            ix += 1
            if ix > len(coll) - 1:
                ix -= 2
                dr = 'bwd'
        else:
            ix -= 1
            if ix < 0:
                ix += 2
                dr = 'fwd'
        n -= 1
    return (ix,dr)

def move_scanner(depth_and_range):
    d,dr,r = depth_and_range
    if 'e' in r: return d,dr,r
    loc = r.index('s')
    r[loc] = '*'
    new_loc,dr = elevator(r,1,loc,dr)
    r[new_loc] = 's'
    return d,dr,r

penalty = []
for i in range(len(all_together)):
    d,dr,r = all_together[i]
    if r[0] == 's':
        penalty.append(d * len(r))
    all_together = list(map(move_scanner,all_together))

sum(penalty)


1844

In [61]:
all_together[0:5]

[(0, 'fwd', ['*', '*', '*', 's']),
 (1, 'fwd', ['*', 's']),
 (2, 'bwd', ['*', 's', '*']),
 (3, 'fwd', ['e']),
 (4, 'fwd', ['*', '*', '*', 's'])]

## Part Two (and improved part one)
I knew there was a simple way using modulo to do the elevator movements, but it just wasn't coming to me when I first tried the puzzle.  The next morning a coworker hinted at the modulo math required to tell if a scanner was at the top, and once I had that, the puzzle became infinitely simpler.  I'm including the original solution to part one (I could easily extend this for part two - at least I think I could) to keep myself honest, since I had a pretty big hint to arrive at the below.

In [62]:
data = [list(map(int,re.findall('\d+',line)))
        for line in open('day13.txt').read().strip().split('\n')]

fwalls = {layer:depth for layer,depth in data}
fwalls_inc_missing = [(layer, fwalls.get(layer,0)) for layer in range(99)]

def is_at_top(i,depth):
    return i % ((depth - 1) * 2) == 0

part1 = sum(l*d if is_at_top(l,d) else 0 for l,d in fwalls_inc_missing)
print(part1)

# part 2
picosecond = 0
safe = False
while not safe:
    safe = not(any(is_at_top(picosecond + layer,depth) 
                   for layer,depth in fwalls.items()))
    if safe:
        print(picosecond)
        break
    picosecond += 1

1844
3897604


# Day 14

## Part One

In [132]:
salt = 'amgozmfv'

def hex_to_bin(hex_digits):
    return ''.join(map(lambda x: bin(int(x,16))[2:].zfill(4),hex_digits))

def process(text,data=list(range(0,256))):
    to_ascii = list(map(ord,text)) + [17, 31, 73, 47, 23]
    pos = 0
    skip = 0
    for i in range(64):
        for l in to_ascii:
            end = pos + l
            if end > len(data):
                end = end % len(data)
                right, left = data[pos:], data[0:end]
                all_rev = list(reversed(right+left))
                data[pos:] = all_rev[0:len(right)]
                data[0:end] = all_rev[len(data[pos:]):]
            else:
                data = data[0:pos] + list(reversed(data[pos:end])) + data[end:]
            pos = pos + skip + l
            if pos > len(data):
                pos = pos % len(data)
            skip += 1
    return data

def knot_hash(text):
    knotted = process(text)
    segmented = [knotted[i:i+16] for i in range(0,len(knotted),16)]
    xord = [reduce(lambda x, y: x ^ y, segment) for segment in segmented]
    to_hex = ''.join([hex(x)[2:].zfill(2) for x in xord])
    return to_hex
              
grid = [hex_to_bin(knot_hash(salt + '-' + str(i))) for i in range(128)]
print(len([sq for row in grid for sq in row if sq == '1']))

8222


In [137]:
def dfs(start,goal=lambda x: x == float('inf'),successors = lambda x: x):
    frontier = [[start]]
    seen = set()
    while frontier:
        path = frontier.pop()
        node = path[-1]
        if tuple(node) not in seen:
            seen.add(tuple(node))
            for s in successors(node):
                if goal(s):
                    return path,len(path)
                frontier.append(path + [s])
    return seen

def neighbors_URDL(grid,coord,is_valid_coord = lambda x: x):
    moves = [vector_process(coord,move) for move in [(-1,0),(0,1),(1,0),(0,-1)]]
    valid_moves = [tuple(m) for m in moves if is_valid_coord(m) 
                   and grid[m[0]][m[1]] == '1']
    return valid_moves
    
def is_valid_coord(coord,xmax=128,ymax=128):
    x,y = coord
    return(0 <= x < xmax) and (0 <= y < ymax) 

def get_clusters(grid,loc):
    return dfs(loc, successors=lambda coord: neighbors_URDL(grid,coord,is_valid_coord))

seen = {frozenset()}
for i in range(len(grid)):
    for j in range(len(grid[i])):
        if (i,j) not in frozenset.union(*seen) and grid[i][j] == '1': 
            seen.add(frozenset(get_clusters(grid,(i,j))))
print(len(seen) - 1) # minus one to account for the empty set


1086


# Day 15

## Part One

In [66]:
A = 699
B = 124

factor_a = 16807
factor_b = 48271
divisor = 2147483647

def generator(val,factor,multiple):
    while True:
        val = val * factor % divisor
        if val % multiple == 0:
            yield val
        
ga = generator(A,factor_a,4)
gb = generator(B,factor_b,8)

total = 0
for i in range(5000000):
    a = bin(next(ga))
    b = bin(next(gb))
    if a[-16:] == b[-16:]:
        total += 1

print(total)

313


# Day 16

## Part One

In [139]:
data = open('day16.txt').read().strip().split(',')
programs = 'abcdefghijklmnop'

In [140]:
print(data[0:10])

['x15/10', 'ph/c', 'x3/14', 's1', 'pf/b', 'x12/0', 'ph/m', 'x14/11', 's4', 'x6/13']


In [141]:
def spin(sx,programs):
    x = int(sx[1:])
    return programs[-x:] + programs[0:len(programs)-x]

def exchange(xab,programs):
    ix_a,ix_b = list(map(int,re.findall('\d+',xab)))
    programs[ix_a], programs[ix_b] = programs[ix_b], programs[ix_a]
    return programs

def partner(pab,programs):
    a,b = pab[1:].split('/')
    ix_a, ix_b = programs.index(a), programs.index(b)
    programs[ix_a], programs[ix_b] = programs[ix_b], programs[ix_a]
    return programs

def process(moves,programs):
    programs = list(programs)
    for m in moves:
        dance_move = m[0]
        if dance_move == 'x':
            programs = exchange(m,programs)
        elif dance_move == 's':
            programs = spin(m,programs)
        elif dance_move == 'p':
            programs = partner(m,programs)
        else:
            raise ValueError('no dance type found that matches!')
    return ''.join(programs)

process(data,programs)

'bkgcdefiholnpmja'

## Part Two
Now we do the same thing, but 1 billion times.  Brute force isn't going to work here, so I am going to check to see if there is a discernible pattern in the dance by looking for cycles.  It turns out, for my input, after 60 iterations we are back where we started.  To be more certain I should test that this pattern holds for a few more cycles of 60, but in the interest of time I just assumed it would and proceeded - thinking I'd come back and try again if it didn't hold. 

Since every 60 dances, we are back at the start, We can just run the dance cycles starting at the beginning, for the remainder of 1 billion divided by 60, or 40 times.

In [142]:
programs = 'abcdefghijklmnop'
seen = set()
while programs not in seen:
    seen.add(programs)
    programs = process(data,programs)

print(len(seen))
print(programs)

60
abcdefghijklmnop


In [143]:
programs = 'abcdefghijklmnop'
for x in range (1000000000%60):
    programs = process(data,programs)
programs

'knmdfoijcbpghlea'

# Day 17

## Part One

In [72]:
steps = 303
nums = [0]
ix = 0
for i in range(1,2018):
    ix = (ix + steps) % len(nums) + 1
    nums = nums[0:ix] + [i] + nums[ix:]
print(nums[ix + 1])

1971


## Part Two

In [73]:
#number after zero will shift to 'i' whenever (i + 3) % len(nums) == 0
length = 1
ix = 0
steps = 303
afterzero = None
for i in range(1,50000001):
    ix = (ix + steps) % length
    if ix == 0:
        afterzero = i
    length += 1
    ix += 1
afterzero

17202899

# Day 18

## Part One

In [179]:
def stream(x,f):
    return x,lambda: stream(f(x),f)

def take(n,stream):
    val, thunk = stream
    if n == 0:
        return val
    else:
        return take(n-1,thunk())
    
take(0,stream(2,lambda x: x**2))

2

In [180]:
data = open('day18.txt').read().strip().split('\n')
data

['set i 31',
 'set a 1',
 'mul p 17',
 'jgz p p',
 'mul a 2',
 'add i -1',
 'jgz i -2',
 'add a -1',
 'set i 127',
 'set p 952',
 'mul p 8505',
 'mod p a',
 'mul p 129749',
 'add p 12345',
 'mod p a',
 'set b p',
 'mod b 10000',
 'snd b',
 'add i -1',
 'jgz i -9',
 'jgz a 3',
 'rcv b',
 'jgz b -1',
 'set f 0',
 'set i 126',
 'rcv a',
 'rcv b',
 'set p a',
 'mul p -1',
 'add p b',
 'jgz p 4',
 'snd a',
 'set a b',
 'jgz 1 3',
 'snd b',
 'set f 1',
 'add i -1',
 'jgz i -11',
 'snd a',
 'jgz f -16',
 'jgz a -19']

In [287]:
sounds = []
all_ops =  [(op,reg,val)for op,reg,*val in [instr.split() for instr in data]]
registers = {reg:0 for op,reg,val in all_ops if reg.isalpha()}

def snd(x,_):
    sounds.append(registers[x])
def set_to(x,y):
    registers[x] = y
def add_to(x,y):
    registers[x] = registers[x] + y
def mul(x,y):
    registers[x] = registers[x] * y
def mod(x,y):
    registers[x] = registers[x] % y
def rcv(x,_):
    if x != 0:
        return sounds[-1],'done'
def jgz(x,y):
    if int(registers.get(x,x)) > 0:
        return y,'not done'

get_op = {'snd':snd,
          'set':set_to,
          'add':add_to,
          'mul':mul,
          'mod':mod,
          'rcv':rcv,
          'jgz':jgz}    

def interpret(registers):
    pos = 0
    while 0 <= pos < len(all_ops):
        op,reg,val = all_ops[pos]
        if val: 
            val = int(registers.get(val[0],val[0]))
        else:
            val = None
        op = get_op[op]
        regval = registers.get(reg,None)
        jmp = op(reg,val)
        if jmp:
            a,maybe_done = jmp
            if maybe_done == 'done':
                return a
            else:
                pos += a
                continue
        pos += 1
interpret(registers)    

4601

In [327]:
def rcv(X,q,register): register[X] = q.pop(0) if q else register[X]
def snd(X,q,register): q.append(int(register.get(X,X)))
def set_(X,Y,register): register[X] = Y
def add(X,Y,register): register[X] += Y
def mul(X,Y,register): register[X] *= Y
def mod(X,Y,register): register[X] %= Y
def jgz(X,Y,register): return Y if int(register.get(X,X)) > 0 else None

all_ops = {'rcv':rcv,'snd':snd,'set':set_,'add':add,'mul':mul,'mod':mod,'jgz':jgz}
programs = [inst.split(' ') for inst in [inst for inst in data]]
# programs =  [(op,reg,val) for op,reg,*val in [instr.split() for instr in data]]
registers0 = {'a':0,'p':0,'i':0,'b':0,'f':0}
registers1 = {'a':0,'p':1,'i':0,'b':0,'f':0}
qs = {0:[],1:[]}

envs = {
        0: {
            'program':0,
            'register':registers0,
            'pos':[0],
            'q':qs[0],
            'otherq':qs[1],
            'sent':[0],
            'finished':[False]
           },

        1: {
            'program':1,
            'register':registers1,
            'pos':[0],
            'q':qs[1],
            'otherq':qs[0],
            'sent':[0],
            'finished':[False]
           }
        }
           
def interpret(program=None,register=None,pos=None,q=None,otherq=None,
              sent=None,finished=None):
    while 0 <= pos[0] < len(programs):
        op,reg,*reg_or_val = programs[pos[0]] # reg is envs[1]['finished'][0]always a reg except for jnz
        func = all_ops[op]
        if reg_or_val:
            reg_or_val = reg_or_val[0]
            val = int(register.get(reg_or_val,reg_or_val))
            maybe_jump = func(reg,val,register)
            if maybe_jump:
                pos[0] += maybe_jump
                continue
        ## if no reg_or_val we know it's either a snd or rcv bc both only take one arg
        elif op == 'snd':
            func(reg,otherq,register)
            sent[0] += 1  
        elif op == 'rcv':
            if not q:
                return 'waiting'
            else:
                func(reg,q,register)
        else:
            raise ValueError('no instruction matching {}'.format(op))
            
        pos[0] += 1
    finished[0] = True

## each needs it's own q, register, position, and sent_times
def run_interpreter():
    while not (envs[0]['finished'][0] and envs[1]['finished'][0]):  
        while not envs[0]['finished'][0]:
            status = interpret(**envs[0])
            if status == 'waiting':
                break
        while not envs[1]['finished'][0]:
            status = interpret(**envs[1])
            if status == 'waiting':
                break
        deadlocked = not(envs[0]['q']) and  not(envs[1]['q'])
        if deadlocked: return envs[1]['sent'][0] 
    return envs[1]['sent'][0]
print(run_interpreter())


6858


# Day 19

## Part One

In [ ]:
data = open('day19.txt').read().strip().split('\n')
data